
# DB Queries with Django ORM

In [1]:
# 3 famous data science, scientific libraries
import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as pd

# Django libraries
from django.db import connection, reset_queries
# from django.db.models import Avg, Count, Min, Max, Q, F, Value as V, Exists, ..
from django.db.models import Value as V, CharField, TextField, FloatField
from django.db.models.functions import Upper, Lower, Substr, \
        Coalesce, Extract, Concat, Round, Ceil, Floor, Cast

#python libraries
from datetime import date

## 1 - Basic CRUD

### 1.1 - Get all elements and pagination

In [2]:
reset_queries()
queryset = Movie.objects.all()
print(connection.queries) # no SQL here, QuerySet object is lazy
queryset

[]


<QuerySet [#5257 - Excuse Me (1915), #6366 - Anton the Terrible (1916), #8351 - The Mysterious Mrs. Musslewhite (1917), #8384 - On the Level (1917), #8642 - The Sunset Trail (1917), #8676 - The Tides of Barnegat (1917), #8981 - The Cruise of the Make-Believes (1918), #9124 - Good Night, Paul (1918), #16251 - Proud Flesh (1925), #19588 - A Woman Against the World (1928), #8079250 - Oppenheimer (2023), #9274 - A Lady's Name (1918), #9403 - Mrs. Leffingwell's Boots (1918), #9475 - A Pair of Silk Stockings (1918), #9492 - A Petticoat Pilot (1918), #9576 - Sauce for the Goose (1918), #9666 - Such a Little Pirate (1918), #9740 - Unclaimed Goods (1918), #9767 - Viviette (1918), #19699 - The Black Watch (1929), '...(remaining elements truncated)...']>

In [3]:
connection.queries

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" LIMIT 21',
  'time': '0.000'}]

In [4]:
reset_queries()
queryset = Movie.objects.all()
movie_slice = queryset[50:100]  # new lazy Query Set, pagination
movie_slice

<QuerySet [#13592 - Shadows (1922), #13611 - Smilin' Through (1922), #13751 - When Love Comes (1922), #16514 - The Wheel (1925), #21015 - Juno and the Paycock (1930), #21128 - Mary (1931), #13891 - Bright Lights of Broadway (1923), #14205 - Little Old New York (1923), #14264 - Maytime (1923), #14572 - Vanity Fair (1923), #14689 - The Average Woman (1924), #14918 - A Fool's Awakening (1924), #15022 - Janice Meredith (1924), #15249 - The Price of a Party (1924), #16048 - Lovers in Quarantine (1925), #16062 - The Mad Marriage (1925), #16093 - The Marriage Whirl (1925), #16127 - The Mountain Eagle (1926), #16230 - The Pleasure Garden (1925), #8079253 - Oppenheimer (2023), '...(remaining elements truncated)...']>

In [5]:
connection.queries

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" LIMIT 21 OFFSET 50',
  'time': '0.000'}]

In [6]:
movies = list(queryset) # a queryset is iterable
print(connection.queries)
print(len(movies), movies[:2], movies[-2:])

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" LIMIT 21 OFFSET 50', 'time': '0.000'}, {'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie"', 'time': '0.016'}]
1202 [#5257 - Excuse Me (1915), #6366 - Anton the Terrible (1916)] [#7959026 - The Mule (2018), #8079248 - Yesterday (2019)]


### 1.2 - Get One element
Only one result: primary key, unique column

In [7]:
m = Movie.objects.get(id=5257)  # pk by column name
m

#5257 - Excuse Me (1915)

In [8]:
m = Movie.objects.get(pk=5257) # pk without the name of the column
m

#5257 - Excuse Me (1915)

In [9]:
try:
    Movie.objects.get(pk=0)
except Movie.DoesNotExist:
    print("No movie with this id")

No movie with this id


### 1.3 - Lifecycle

In [10]:
reset_queries()
m = Movie(title="Oppenheimer", year=2023)
print("movie id:", m.id) 
m.save()
print("movie id:", m.id)

movie id: None
movie id: 8079272


In [11]:
connection.queries

[{'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Oppenheimer\', 2023, NULL, NULL, NULL, NULL, NULL) RETURNING "movieapp_movie"."id"',
  'time': '0.000'}]

In [12]:
m.delete()

(1, {'movieapp.Movie': 1})

In [13]:
connection.queries

[{'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Oppenheimer\', 2023, NULL, NULL, NULL, NULL, NULL) RETURNING "movieapp_movie"."id"',
  'time': '0.000'},
 {'sql': 'DELETE FROM "movieapp_movie" WHERE "movieapp_movie"."id" IN (8079272)',
  'time': '0.000'}]

In [14]:
reset_queries()
movieOppenheimer = Movie.objects.create(title="Oppenheimer", year=2023, duration=180, pg='R')
print("movie id:", movieOppenheimer.id)
connection.queries

movie id: 8079273


[{'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Oppenheimer\', 2023, 180, \'R\', NULL, NULL, NULL) RETURNING "movieapp_movie"."id"',
  'time': '0.000'}]

In [15]:
reset_queries()
movieOppenheimer.synopsis = "The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb."
movieOppenheimer.save()
connection.queries

[{'sql': 'UPDATE "movieapp_movie" SET "title" = \'Oppenheimer\', "year" = 2023, "duration" = 180, "pg" = \'R\', "synopsis" = \'The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb.\', "poster_uri" = NULL, "director_id" = NULL WHERE "movieapp_movie"."id" = 8079273',
  'time': '0.000'}]

## 2 - ManyToOne CRUD

In [16]:
reset_queries()
m = Movie.objects.get(pk=7959026)
print(connection.queries)
m

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" WHERE "movieapp_movie"."id" = 7959026 LIMIT 21', 'time': '0.000'}]


#7959026 - The Mule (2018)

In [17]:
m.director_id

142

In [18]:
reset_queries()
d = m.director # lazy attribute is filled here (query SQL)
print(connection.queries)
d

[{'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."id" = 142 LIMIT 21', 'time': '0.000'}]


#142 - Clint Eastwood

In [19]:
print(d)

#142 - Clint Eastwood


In [20]:
nolansQuerySet = Person.objects.filter(name = "Christopher Nolan")
nolans = list(nolansQuerySet)
nolans

[#634240 - Christopher Nolan]

In [21]:
reset_queries()
nolan = nolans[0]
movieOppenheimer.director = nolan # set object relationship
movieOppenheimer.save()
connection.queries

[{'sql': 'UPDATE "movieapp_movie" SET "title" = \'Oppenheimer\', "year" = 2023, "duration" = 180, "pg" = \'R\', "synopsis" = \'The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb.\', "poster_uri" = NULL, "director_id" = 634240 WHERE "movieapp_movie"."id" = 8079273',
  'time': '0.016'}]

In [22]:
# create movie BArbie, 2023 by Greta Gerwig Born August 4, 1983
reset_queries()
greta = Person.objects.create(name="Greta Gerwig", birthdate=date(1983,8,4))
movieBarbie = Movie.objects.create(title="Barbie", year=2023, director=greta)
connection.queries

[{'sql': 'INSERT INTO "person" ("name", "birthdate") VALUES (\'Greta Gerwig\', \'1983-08-04\'::date) RETURNING "person"."id"',
  'time': '0.000'},
 {'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Barbie\', 2023, NULL, NULL, NULL, NULL, 11903880) RETURNING "movieapp_movie"."id"',
  'time': '0.000'}]

## 3 - Queries

### 3.1. Filters (Where)

In [23]:
# exact predicate (=)
Movie.objects.filter(title='The Man Who Knew Too Much')

<QuerySet [#25452 - The Man Who Knew Too Much (1934), #49470 - The Man Who Knew Too Much (1956)]>

In [24]:
# 2 predicates AND
Movie.objects.filter(title='The Man Who Knew Too Much', year=1956)

<QuerySet [#49470 - The Man Who Knew Too Much (1956)]>

In [25]:
# iexact = exact case insensitive
Movie.objects.filter(title__iexact="the man who knew too much")

<QuerySet [#25452 - The Man Who Knew Too Much (1934), #49470 - The Man Who Knew Too Much (1956)]>

In [26]:
# (i)startswith, (i)endswith, (i)contains
Movie.objects.filter(title__istartswith="the man")

<QuerySet [#20142 - The Manxman (1929), #24293 - The Man from Monterey (1933), #25452 - The Man Who Knew Too Much (1934), #25455 - The Man from Utah (1934), #49470 - The Man Who Knew Too Much (1956), #56217 - The Man Who Shot Liberty Valance (1962), #71807 - The Man with the Golden Gun (1974), #73341 - The Man Who Would Be King (1975), #89543 - The Man with One Red Shoe (1985), #243133 - The Man Who Wasn't There (2001)]>

In [27]:
Movie.objects.filter(title__icontains="star")

<QuerySet [#25830 - The Star Packer (1934), #49793 - Star in the Dust (1956), #76759 - Star Wars: Episode IV - A New Hope (1977), #80684 - Star Wars: Episode V - The Empire Strikes Back (1980), #86190 - Star Wars: Episode VI - Return of the Jedi (1983), #95405 - Starry is the Night (1988), #2488496 - Star Wars: Episode VII - The Force Awakens (2015), #2527338 - Star Wars: Episode IX - The Rise of Skywalker (2019)]>

In [28]:
# (i)regex: regular expression
Movie.objects.filter(title__iregex=r"(^| )star( |$)")

<QuerySet [#25830 - The Star Packer (1934), #49793 - Star in the Dust (1956), #76759 - Star Wars: Episode IV - A New Hope (1977), #80684 - Star Wars: Episode V - The Empire Strikes Back (1980), #86190 - Star Wars: Episode VI - Return of the Jedi (1983), #2488496 - Star Wars: Episode VII - The Force Awakens (2015), #2527338 - Star Wars: Episode IX - The Rise of Skywalker (2019)]>

In [29]:
# comparison operator: gt >, gte >=, lt <, lte <=
Movie.objects.filter(year__gt=2018)

<QuerySet [#8079250 - Oppenheimer (2023), #8079251 - Oppenheimer (2023), #8079253 - Oppenheimer (2023), #8079255 - Oppenheimer (2023), #8079256 - Barbie (2023), #8079258 - Oppenheimer (2023), #8079259 - Barbie (2023), #8079261 - Oppenheimer (2023), #8079262 - Barbie (2023), #8079265 - Barbie (2023), #8079264 - Oppenheimer (2023), #8079268 - Barbie (2023), #8079267 - Oppenheimer (2023), #8079270 - Oppenheimer (2023), #8079271 - Barbie (2023), #8079273 - Oppenheimer (2023), #8079274 - Barbie (2023), #1025100 - Gemini Man (2019), #1206885 - Rambo: Last Blood (2019), #1302006 - The Irishman (2019), '...(remaining elements truncated)...']>

In [30]:
# range: between
movies = list(Movie.objects
              .exclude(year=1982)
              .filter(year__range=(1980,1989))
              .order_by("year", "title")
             )
movies

[#80377 - Any Which Way You Can (1980),
 #80472 - Bronco Billy (1980),
 #80661 - Dressed to Kill (1980),
 #80850 - He Knows You're Alone (1980),
 #82880 - Il pap'occhio (1980),
 #76580 - Qian zuo guai (1980),
 #81398 - Raging Bull (1980),
 #80684 - Star Wars: Episode V - The Empire Strikes Back (1980),
 #80455 - The Blues Brothers (1980),
 #80907 - The Hunter (1980),
 #80031 - Tom Horn (1980),
 #82085 - Blow Out (1981),
 #82398 - For Your Eyes Only (1981),
 #82535 - Hua ji shi dai (1981),
 #82817 - Nighthawks (1981),
 #82869 - Outland (1981),
 #82910 - Piranha II: The Spawning (1981),
 #82971 - Raiders of the Lost Ark (1981),
 #83907 - The Evil Dead (1981),
 #81633 - Time Bandits (1981),
 #83284 - Victory (1981),
 #173893 - Hefty's (1983),
 #86006 - Never Say Never Again (1983),
 #86034 - Octopussy (1983),
 #86250 - Scarface (1983),
 #86190 - Star Wars: Episode VI - Return of the Jedi (1983),
 #86361 - Staying Alive (1983),
 #86383 - Sudden Impact (1983),
 #86491 - Twilight Zone: The M

In [31]:
Movie.objects.filter(duration__isnull=True)

<QuerySet [#8079250 - Oppenheimer (2023), #13444 - Number 13 (1922), #8079256 - Barbie (2023), #8079259 - Barbie (2023), #8079262 - Barbie (2023), #8079265 - Barbie (2023), #8079268 - Barbie (2023), #8079271 - Barbie (2023), #8079274 - Barbie (2023), #77667 - Ha luo, ye gui ren (1978), #173893 - Hefty's (1983), #185446 - Gu lian hua (1985), #4824394 - Terminator Genisys: The YouTube Chronicles (2015), #6344712 - The Square Root (1969)]>

In [32]:
date1 = date(1930,1,1)
date2 = date(1930,12,31)
Person.objects.filter(birthdate__range=(date1,date2))

<QuerySet [#125 - Sean Connery, #142 - Clint Eastwood, #432 - Gene Hackman, #537 - Steve McQueen, #728 - Mario Adorf, #1262 - Ben Gazzara, #1321 - Richard Harris, #1335 - Tippi Hedren, #1687 - Gena Rowlands, #1703 - Maximilian Schell, #1792 - Rod Taylor, #1822 - Robert Wagner, #3679 - Kate Reid, #4877 - Roy Edward Disney, #5162 - Robert Loggia, #5196 - Paul Mazursky, #43484 - Jack Axelrod, #53364 - Chris Barber, #58972 - Dominic Barto, #60789 - Charles Bateman, '...(remaining elements truncated)...']>

In [33]:
reset_queries()
Person.objects.filter(birthdate__year=1930)

<QuerySet [#125 - Sean Connery, #142 - Clint Eastwood, #432 - Gene Hackman, #537 - Steve McQueen, #728 - Mario Adorf, #1262 - Ben Gazzara, #1321 - Richard Harris, #1335 - Tippi Hedren, #1687 - Gena Rowlands, #1703 - Maximilian Schell, #1792 - Rod Taylor, #1822 - Robert Wagner, #3679 - Kate Reid, #4877 - Roy Edward Disney, #5162 - Robert Loggia, #5196 - Paul Mazursky, #43484 - Jack Axelrod, #53364 - Chris Barber, #58972 - Dominic Barto, #60789 - Charles Bateman, '...(remaining elements truncated)...']>

In [34]:
connection.queries

[{'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."birthdate" BETWEEN \'1930-01-01\'::date AND \'1930-12-31\'::date LIMIT 21',
  'time': '0.000'}]

In [35]:
reset_queries()
Person.objects.annotate(year=Extract('birthdate','year')).filter(year=1930)

<QuerySet [#125 - Sean Connery, #142 - Clint Eastwood, #432 - Gene Hackman, #537 - Steve McQueen, #728 - Mario Adorf, #1262 - Ben Gazzara, #1321 - Richard Harris, #1335 - Tippi Hedren, #1687 - Gena Rowlands, #1703 - Maximilian Schell, #1792 - Rod Taylor, #1822 - Robert Wagner, #3679 - Kate Reid, #4877 - Roy Edward Disney, #5162 - Robert Loggia, #5196 - Paul Mazursky, #43484 - Jack Axelrod, #53364 - Chris Barber, #58972 - Dominic Barto, #60789 - Charles Bateman, '...(remaining elements truncated)...']>

In [36]:
connection.queries

[{'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate", EXTRACT(YEAR FROM "person"."birthdate") AS "year" FROM "person" WHERE EXTRACT(YEAR FROM "person"."birthdate") = 1930 LIMIT 21',
  'time': '0.000'}]

### 3.2 - Projections

In [37]:
Movie.objects.values('year')

<QuerySet [{'year': 1915}, {'year': 1916}, {'year': 1917}, {'year': 1917}, {'year': 1917}, {'year': 1917}, {'year': 1918}, {'year': 1918}, {'year': 1925}, {'year': 1928}, {'year': 2023}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1929}, '...(remaining elements truncated)...']>

In [38]:
Movie.objects.filter(title__icontains='star').values('year')

<QuerySet [{'year': 1934}, {'year': 1956}, {'year': 1977}, {'year': 1980}, {'year': 1983}, {'year': 1988}, {'year': 2015}, {'year': 2019}]>

In [39]:
Movie.objects.filter(title__icontains='star').values('title','year')

<QuerySet [{'title': 'The Star Packer', 'year': 1934}, {'title': 'Star in the Dust', 'year': 1956}, {'title': 'Star Wars: Episode IV - A New Hope', 'year': 1977}, {'title': 'Star Wars: Episode V - The Empire Strikes Back', 'year': 1980}, {'title': 'Star Wars: Episode VI - Return of the Jedi', 'year': 1983}, {'title': 'Starry is the Night', 'year': 1988}, {'title': 'Star Wars: Episode VII - The Force Awakens', 'year': 2015}, {'title': 'Star Wars: Episode IX - The Rise of Skywalker', 'year': 2019}]>

In [40]:
# distinct
years = list(Movie.objects.values('year').distinct())
len(years)

107

In [41]:
reset_queries()
q = Person.objects \
        .filter(birthdate__isnull=False) \
        .annotate(year=Extract('birthdate', 'year')) \
        .values('name','year') \
        .order_by('-year','name')
q

<QuerySet [{'name': 'Gary the Dog', 'year': 2012}, {'name': 'Temirlan Blaev', 'year': 2007}, {'name': 'Erin Kellyman', 'year': 1998}, {'name': 'Ross Beadman', 'year': 1998}, {'name': 'Alex Knoll', 'year': 1994}, {'name': 'Chelsea Mather', 'year': 1994}, {'name': 'Keira Wingate', 'year': 1994}, {'name': 'Jett Lucas', 'year': 1993}, {'name': "Helena Collins O'Connor", 'year': 1992}, {'name': 'Chris Adams', 'year': 1990}, {'name': 'Jennifer Lawrence', 'year': 1990}, {'name': 'Keisha Castle-Hughes', 'year': 1990}, {'name': 'Sam Wilkinson', 'year': 1990}, {'name': 'Dalton Abbott', 'year': 1989}, {'name': 'Jake Lloyd', 'year': 1989}, {'name': 'James Henri-Thomas', 'year': 1989}, {'name': 'Samantha Colley', 'year': 1989}, {'name': 'Seth Adkins', 'year': 1989}, {'name': 'Bobby Edner', 'year': 1988}, {'name': 'Chelsea Hamill', 'year': 1988}, '...(remaining elements truncated)...']>

In [42]:
connection.queries

[{'sql': 'SELECT "person"."name", EXTRACT(YEAR FROM "person"."birthdate") AS "year" FROM "person" WHERE "person"."birthdate" IS NOT NULL ORDER BY 2 DESC, "person"."name" ASC LIMIT 21',
  'time': '0.016'}]

### 3.3 - Special functions Q, V, F
- **Q** = wrap a condition ; usefull with Or, Xor combination
- **F** = use value of a field ; usefull with comparison/operators between 2 columns
- **Value as V** = wrap a value for disambiguation purpose with column name

In [43]:
# .filter(duration__isnull=True) \
q = Movie.objects \
    .filter(year=2023) \
    .annotate(duration2=Coalesce('duration', 0)) \
    .values('title', 'duration2')
q

<QuerySet [{'title': 'Oppenheimer', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}, {'title': 'Barbie', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}]>

In [44]:
Movie.objects \
    .filter(year=1984) \
    .annotate(long_title=Concat('title', Cast('year', CharField()))) \
    .values('long_title')

<QuerySet [{'long_title': 'Sword of the Valiant: The Legend of Sir Gawain and the Green Knight1984'}, {'long_title': 'Bachelor Party1984'}, {'long_title': 'Blood Simple1984'}, {'long_title': 'Body Double1984'}, {'long_title': "Breakin'1984"}, {'long_title': 'City Heat1984'}, {'long_title': 'Conan the Destroyer1984'}, {'long_title': 'Gremlins1984'}, {'long_title': 'Indiana Jones and the Temple of Doom1984'}, {'long_title': 'Missing in Action1984'}, {'long_title': 'Monaco Forever1984'}, {'long_title': 'Rhinestone1984'}, {'long_title': 'Splash1984'}, {'long_title': 'The Terminator1984'}, {'long_title': 'Tightrope1984'}, {'long_title': 'Heroes Shed No Tears1984'}]>

In [45]:
q = Movie.objects \
    .filter(year=1984) \
    .annotate(long_title=Concat(
            'title', 
            V(' ('), 
            Cast('year', CharField()), 
            V(')'
        ))) \
    .values('title', 'year', 'long_title')
res = list(q)
for m in res:
    print(m['long_title'])

Sword of the Valiant: The Legend of Sir Gawain and the Green Knight (1984)
Bachelor Party (1984)
Blood Simple (1984)
Body Double (1984)
Breakin' (1984)
City Heat (1984)
Conan the Destroyer (1984)
Gremlins (1984)
Indiana Jones and the Temple of Doom (1984)
Missing in Action (1984)
Monaco Forever (1984)
Rhinestone (1984)
Splash (1984)
The Terminator (1984)
Tightrope (1984)
Heroes Shed No Tears (1984)


In [46]:
df = pd.DataFrame(res)
df

,title,year,long_title
0,Sword of the Valiant: The Legend of Sir Gawain...,1984,Sword of the Valiant: The Legend of Sir Gawain...
1,Bachelor Party,1984,Bachelor Party (1984)
2,Blood Simple,1984,Blood Simple (1984)
3,Body Double,1984,Body Double (1984)
4,Breakin',1984,Breakin' (1984)
5,City Heat,1984,City Heat (1984)
6,Conan the Destroyer,1984,Conan the Destroyer (1984)
7,Gremlins,1984,Gremlins (1984)
8,Indiana Jones and the Temple of Doom,1984,Indiana Jones and the Temple of Doom (1984)
9,Missing in Action,1984,Missing in Action (1984)


In [47]:
# AND with conditions
Movie.objects.filter(year=1984, title__icontains='terminator')

<QuerySet [#88247 - The Terminator (1984)]>

In [48]:
# AND &, OR |, XOR ^, NOT ~
for m in Movie.objects.filter(Q(year=1984) | Q(title__icontains='terminator')):
    print(m)


#84750 - Sword of the Valiant: The Legend of Sir Gawain and the Green Knight (1984)
#86927 - Bachelor Party (1984)
#86979 - Blood Simple (1984)
#86984 - Body Double (1984)
#86998 - Breakin' (1984)
#87062 - City Heat (1984)
#87078 - Conan the Destroyer (1984)
#87363 - Gremlins (1984)
#87469 - Indiana Jones and the Temple of Doom (1984)
#87727 - Missing in Action (1984)
#87739 - Monaco Forever (1984)
#88001 - Rhinestone (1984)
#88161 - Splash (1984)
#88247 - The Terminator (1984)
#88272 - Tightrope (1984)
#92264 - Heroes Shed No Tears (1984)
#103064 - Terminator 2: Judgment Day (1991)
#181852 - Terminator 3: Rise of the Machines (2003)
#1340138 - Terminator Genisys (2015)
#4824394 - Terminator Genisys: The YouTube Chronicles (2015)
#6450804 - Terminator: Dark Fate (2019)


In [49]:
Movie.objects.filter(~Q(year=1984)).count(), Movie.objects.filter(year=1984).count(), Movie.objects.count()

(1188, 16, 1204)

In [50]:
# same technique in pandas (and numpy)
df[df.title.str.contains('terminator', case=False) & (df.year == 1984)]

,title,year,long_title
13,The Terminator,1984,The Terminator (1984)


In [51]:
for m in Movie.objects.annotate(dummy=F('year') + 100) \
        .values('title', 'dummy')[:10]:
    print(m)

{'title': 'Excuse Me', 'dummy': 2015}
{'title': 'Anton the Terrible', 'dummy': 2016}
{'title': 'The Mysterious Mrs. Musslewhite', 'dummy': 2017}
{'title': 'On the Level', 'dummy': 2017}
{'title': 'The Sunset Trail', 'dummy': 2017}
{'title': 'The Tides of Barnegat', 'dummy': 2017}
{'title': 'The Cruise of the Make-Believes', 'dummy': 2018}
{'title': 'Good Night, Paul', 'dummy': 2018}
{'title': 'Proud Flesh', 'dummy': 2025}
{'title': 'A Woman Against the World', 'dummy': 2028}


### 3.4 - Navigate through relationship

In [52]:
clint = Person.objects.get(name='Clint Eastwood')
clint

#142 - Clint Eastwood

In [53]:
# movie_set (director only) or directedMovies (with related name): reverse navigation then add classic methods of manager objects 
# (all, get, filter, annotate, ...)
print("Clint Eastwood as director")
for m in clint.directedMovies.order_by('-year'):
    print(m)
print()
print("Clint Eastwood as actor")
for m in clint.playedMovies.order_by('-year'):
    print(m)

Clint Eastwood as director
#3513548 - Richard Jewell (2019)
#7959026 - The Mule (2018)
#6802308 - The 15:17 to Paris (2018)
#3263904 - Sully (2016)
#2179136 - American Sniper (2014)
#1742044 - Jersey Boys (2014)
#1616195 - J. Edgar (2011)
#1212419 - Hereafter (2010)
#1057500 - Invictus (2009)
#1205489 - Gran Torino (2008)
#824747 - Changeling (2008)
#418689 - Flags of Our Fathers (2006)
#498380 - Letters from Iwo Jima (2006)
#405159 - Million Dollar Baby (2004)
#327056 - Mystic River (2003)
#309377 - Blood Work (2002)
#186566 - Space Cowboys (2000)
#139668 - True Crime (1999)
#119668 - Midnight in the Garden of Good and Evil (1997)
#118548 - Absolute Power (1997)
#112579 - The Bridges of Madison County (1995)
#107808 - A Perfect World (1993)
#105695 - Unforgiven (1992)
#100928 - White Hunter Black Heart (1990)
#100514 - The Rookie (1990)
#94747 - Bird (1988)
#91187 - Heartbreak Ridge (1986)
#89767 - Pale Rider (1985)
#86383 - Sudden Impact (1983)
#84088 - Honkytonk Man (1982)
#83943 - 

In [54]:
Movie.objects.filter(director=clint)

<QuerySet [#67588 - Play Misty for Me (1971), #68699 - High Plains Drifter (1973), #69822 - Breezy (1973), #72926 - The Eiger Sanction (1975), #75029 - The Outlaw Josey Wales (1976), #76070 - The Gauntlet (1977), #80472 - Bronco Billy (1980), #83943 - Firefox (1982), #84088 - Honkytonk Man (1982), #86383 - Sudden Impact (1983), #89767 - Pale Rider (1985), #91187 - Heartbreak Ridge (1986), #94747 - Bird (1988), #100514 - The Rookie (1990), #100928 - White Hunter Black Heart (1990), #105695 - Unforgiven (1992), #107808 - A Perfect World (1993), #112579 - The Bridges of Madison County (1995), #118548 - Absolute Power (1997), #119668 - Midnight in the Garden of Good and Evil (1997), '...(remaining elements truncated)...']>

In [55]:
reset_queries()
for m in Movie.objects.filter(director__name__istartswith='steve'):
    print(m.title, m.director)

Firelight #229 - Steven Spielberg
The Sugarland Express #229 - Steven Spielberg
Capone #142587 - Steve Carver
Jaws #229 - Steven Spielberg
Close Encounters of the Third Kind #229 - Steven Spielberg
1941 #229 - Steven Spielberg
Raiders of the Lost Ark #229 - Steven Spielberg
E.T. the Extra-Terrestrial #229 - Steven Spielberg
Indiana Jones and the Temple of Doom #229 - Steven Spielberg
The Color Purple #229 - Steven Spielberg
Empire of the Sun #229 - Steven Spielberg
Always #229 - Steven Spielberg
Indiana Jones and the Last Crusade #229 - Steven Spielberg
The Visionary #229 - Steven Spielberg
Hook #229 - Steven Spielberg
Jurassic Park #229 - Steven Spielberg
Schindler's List #229 - Steven Spielberg
Street Fighter #211823 - Steven E. de Souza
Amistad #229 - Steven Spielberg
The Lost World: Jurassic Park #229 - Steven Spielberg
Saving Private Ryan #229 - Steven Spielberg
Michael Jackson: Video Greatest Hits - HIStory #6625 - Steve Barron
Little Nicky #109359 - Steven Brill
Minority Report 

In [56]:
# NB: N+1 queries to be optimized
connection.queries

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" INNER JOIN "person" ON ("movieapp_movie"."director_id" = "person"."id") WHERE UPPER("person"."name"::text) LIKE UPPER(\'steve%\')',
  'time': '0.016'},
 {'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."id" = 229 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."id" = 229 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."id" = 142587 LIMIT 21',
  'time': '0.015'},
 {'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."id" = 229 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "person"

### 3.5 - Group By + Aggregate functions 

In [57]:
# NB: values clause gives grouping criteria + fields projected
for row in Movie.objects.values('year').annotate(nb=Count('id')).order_by('-year')[:10]:
    print(row)

{'year': 2023, 'nb': 17}
{'year': 2020, 'nb': 4}
{'year': 2019, 'nb': 21}
{'year': 2018, 'nb': 16}
{'year': 2017, 'nb': 18}
{'year': 2016, 'nb': 13}
{'year': 2015, 'nb': 22}
{'year': 2014, 'nb': 16}
{'year': 2013, 'nb': 25}
{'year': 2012, 'nb': 21}


In [58]:
Movie.objects.aggregate(nb=Count('id'), year_min=Min('year'), year_max=Max('year'), duration_total=Sum('duration'))

{'nb': 1204, 'year_min': 1915, 'year_max': 2023, 'duration_total': 128815}

In [59]:
# stats by director: nb movies, first year, last year, total duration
# exclude clause optimize query with INNER JOIN instead of LEFT JOIN
Movie.objects \
    .exclude(director__isnull=True) \
    .values('director__id', 'director__name').annotate(nb=Count('id'), year_min=Min('year'), year_max=Max('year'), duration_total=Sum('duration'))

<QuerySet [{'director__id': 33, 'director__name': 'Alfred Hitchcock', 'nb': 55, 'year_min': 1922, 'year_max': 1976, 'duration_total': 5430}, {'director__id': 41, 'director__name': 'Akira Kurosawa', 'nb': 1, 'year_min': 1990, 'year_max': 1990, 'duration_total': 119}, {'director__id': 78, 'director__name': 'John Wayne', 'nb': 1, 'year_min': 1960, 'year_max': 1960, 'duration_total': 162}, {'director__id': 95, 'director__name': 'Woody Allen', 'nb': 3, 'year_min': 1971, 'year_max': 2015, 'duration_total': 301}, {'director__id': 116, 'director__name': 'James Cameron', 'nb': 8, 'year_min': 1981, 'year_max': 2009, 'duration_total': 1143}, {'director__id': 142, 'director__name': 'Clint Eastwood', 'nb': 38, 'year_min': 1971, 'year_max': 2019, 'duration_total': 4768}, {'director__id': 158, 'director__name': 'Tom Hanks', 'nb': 2, 'year_min': 1996, 'year_max': 2011, 'duration_total': 206}, {'director__id': 165, 'director__name': 'Ron Howard', 'nb': 6, 'year_min': 1984, 'year_max': 2016, 'duration_t

In [60]:
connection.queries[-1]

{'sql': 'SELECT "movieapp_movie"."director_id", "person"."name", COUNT("movieapp_movie"."id") AS "nb", MIN("movieapp_movie"."year") AS "year_min", MAX("movieapp_movie"."year") AS "year_max", SUM("movieapp_movie"."duration") AS "duration_total" FROM "movieapp_movie" INNER JOIN "person" ON ("movieapp_movie"."director_id" = "person"."id") WHERE NOT ("movieapp_movie"."director_id" IS NULL) GROUP BY "movieapp_movie"."director_id", "person"."name" LIMIT 21',
 'time': '0.000'}

In [61]:
# same query with Having clause
threshold = 10
for row in Movie.objects \
        .exclude(director__isnull=True) \
        .values('director__id', 'director__name') \
        .annotate(nb=Count('id'), year_min=Min('year'), year_max=Max('year'), duration_total=Sum('duration')) \
        .filter(nb__gte=threshold) \
        .order_by('-nb'):
    print(row)

{'director__id': 33, 'director__name': 'Alfred Hitchcock', 'nb': 55, 'year_min': 1922, 'year_max': 1976, 'duration_total': 5430}
{'director__id': 142, 'director__name': 'Clint Eastwood', 'nb': 38, 'year_min': 1971, 'year_max': 2019, 'duration_total': 4768}
{'director__id': 229, 'director__name': 'Steven Spielberg', 'nb': 33, 'year_min': 1964, 'year_max': 2018, 'duration_total': 4421}
{'director__id': 247, 'director__name': 'John Woo', 'nb': 32, 'year_min': 1974, 'year_max': 2017, 'duration_total': 3491}
{'director__id': 361, 'director__name': 'Brian De Palma', 'nb': 30, 'year_min': 1968, 'year_max': 2019, 'duration_total': 3138}
{'director__id': 217, 'director__name': 'Martin Scorsese', 'nb': 25, 'year_min': 1967, 'year_max': 2019, 'duration_total': 3437}
{'director__id': 631, 'director__name': 'Ridley Scott', 'nb': 25, 'year_min': 1977, 'year_max': 2017, 'duration_total': 3218}
{'director__id': 406, 'director__name': 'John Ford', 'nb': 20, 'year_min': 1927, 'year_max': 1963, 'duration

In [62]:
connection.queries[-1]

{'sql': 'SELECT "movieapp_movie"."director_id", "person"."name", COUNT("movieapp_movie"."id") AS "nb", MIN("movieapp_movie"."year") AS "year_min", MAX("movieapp_movie"."year") AS "year_max", SUM("movieapp_movie"."duration") AS "duration_total" FROM "movieapp_movie" INNER JOIN "person" ON ("movieapp_movie"."director_id" = "person"."id") WHERE NOT ("movieapp_movie"."director_id" IS NULL) GROUP BY "movieapp_movie"."director_id", "person"."name" HAVING COUNT("movieapp_movie"."id") >= 10 ORDER BY 3 DESC',
 'time': '0.000'}

## 4 - ManyToMany

### 4.1 - without role

In [63]:
movieOppenheimer

#8079273 - Oppenheimer (2023)

In [64]:
#Stars: Cillian Murphy Emily Blunt Matt Damon
mat = Person.objects.filter(name__icontains='Matt Damon')[0]
emily = Person.objects.filter(name__icontains='Emily Blunt')[0]
cylian = Person.objects.filter(name__icontains='Cillian Murphy')[0]

In [65]:
# add or set actors: actors.add, actors.set, actors.remove
reset_queries()
movieOppenheimer.actors.add(mat, emily, cylian)
movieOppenheimer.save()

In [66]:
connection.queries

[{'sql': 'BEGIN', 'time': '0.000'},
 {'sql': 'SELECT "play"."actor_id" FROM "play" WHERE ("play"."actor_id" IN (1289434, 354, 614165) AND "play"."movie_id" = 8079273)',
  'time': '0.000'},
 {'sql': 'INSERT INTO "play" ("movie_id", "actor_id", "role") VALUES (8079273, 1289434, NULL), (8079273, 354, NULL), (8079273, 614165, NULL) RETURNING "play"."id"',
  'time': '0.000'},
 {'sql': 'COMMIT', 'time': '0.016'},
 {'sql': 'UPDATE "movieapp_movie" SET "title" = \'Oppenheimer\', "year" = 2023, "duration" = 180, "pg" = \'R\', "synopsis" = \'The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb.\', "poster_uri" = NULL, "director_id" = 634240 WHERE "movieapp_movie"."id" = 8079273',
  'time': '0.000'}]

In [67]:
movieOppenheimer.actors.order_by('name')

<QuerySet [#614165 - Cillian Murphy, #1289434 - Emily Blunt, #354 - Matt Damon]>

### 4.2 - With role
To add actors with field role use following methods:
- movieBarbie.actors.add, movieBarbie.actors.set, movieBarbie.actors.create with through_defaults arg
- Play.objects.create

In [68]:
movieBarbie

#8079274 - Barbie (2023)

In [69]:
margot = Person.objects.filter(name__icontains='Margot Robbie')[0]
ryan = Person.objects.filter(name__icontains='ryan gosling')[0]

In [70]:
movieBarbie.actors.add(margot, through_defaults={'role':'Barbie'})
movieBarbie.save()
connection.queries[-2]

{'sql': 'COMMIT', 'time': '0.000'}

In [71]:
Play.objects.create(movie=movieBarbie, actor=ryan, role='Ken')

<Play: Play object (66562)>

In [72]:
connection.queries[-5:]

[{'sql': 'SELECT "play"."actor_id" FROM "play" WHERE ("play"."actor_id" IN (3053338) AND "play"."movie_id" = 8079274)',
  'time': '0.016'},
 {'sql': 'INSERT INTO "play" ("movie_id", "actor_id", "role") VALUES (8079274, 3053338, \'Barbie\') RETURNING "play"."id"',
  'time': '0.000'},
 {'sql': 'COMMIT', 'time': '0.000'},
 {'sql': 'UPDATE "movieapp_movie" SET "title" = \'Barbie\', "year" = 2023, "duration" = NULL, "pg" = NULL, "synopsis" = NULL, "poster_uri" = NULL, "director_id" = 11903880 WHERE "movieapp_movie"."id" = 8079274',
  'time': '0.000'},
 {'sql': 'INSERT INTO "play" ("movie_id", "actor_id", "role") VALUES (8079274, 331516, \'Ken\') RETURNING "play"."id"',
  'time': '0.000'}]

In [73]:
# actors with notion ofplay/role transparent
movieBarbie.actors.all()

<QuerySet [#3053338 - Margot Robbie, #331516 - Ryan Gosling]>

In [76]:
# actors through play table with role
# NB: play_set manager name can be customized with arg related_name in class Play mapping
# for p in movieBarbie.play_set.all():
for p in movieBarbie.actorsWithRole.all():
     print(p.actor, ' -> ', p.role)

#3053338 - Margot Robbie  ->  Barbie
#331516 - Ryan Gosling  ->  Ken


In [ ]:
for p in Play.objects.filter(movie=movieBarbie):
    print(p.actor, ' -> ', p.role)